In [188]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [189]:
pkl_file = open('df.pkl', 'rb')
df = pickle.load(pkl_file)
pkl_file.close() 

In [302]:
features = [
#             'top_elev_(ft)', 
#             'bottom_elev_(ft)', 
#             'vert_rise_(ft)', 
#             'slope_length_(ft)', 
#             'avg_width_(ft)', 
#             'slope_area_(acres)', 
            'avg_grade_(%)', 
            'max_grade_(%)', 
            'groomed']

X = df[features].values

In [303]:
X

array([[ 50.,  69.,   0.],
       [ 55.,  66.,   0.],
       [ 42.,  76.,   0.],
       ..., 
       [  8.,  11.,   1.],
       [ 12.,  12.,   1.],
       [ 15.,  12.,   1.]])

In [304]:
def cos_sim_recommendations(trail_name, resort_name, X, n=5, resort=None):
    index = df.index[(df['trail_name'] == trail_name) & (df['resort'] == resort_name)][0]
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)
    rec_index = np.argsort(cs)[0][::-1][1:]
    ordered_df = df.loc[rec_index]
    if resort:
        ordered_df = ordered_df[ordered_df['resort'] == resort]
    rec_df = ordered_df.head(n)
    orig_row = df.loc[[index]].rename(lambda x: 'original')
    total = pd.concat((orig_row,rec_df))
    return total

In [305]:
ss = StandardScaler()
X_trans = ss.fit_transform(X)

In [306]:
X_trans

array([[ 1.7960483 ,  1.3829293 , -0.81952402],
       [ 2.18280168,  1.22823665, -0.81952402],
       [ 1.1772429 ,  1.74387879, -0.81952402],
       ..., 
       [-1.45268006, -1.6077951 ,  1.22022049],
       [-1.14327736, -1.55623089,  1.22022049],
       [-0.91122533, -1.55623089,  1.22022049]])

In [307]:
cos_sim_recommendations('Sorensen Park','Winter Park',X_trans,n=5)

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Sorensen Park,9150.0,9090.0,60.0,1067.43,114,4.07,6.0,8.0,Beginner,Winter Park,CO,1,green,1,1
848,Sorensen Park,9150.0,9090.0,60.0,1067.43,114,4.07,6.0,8.0,Beginner,Winter Park,CO,1,green,1,1
821,School Yard,6799.0,6732.0,66.0,980.00,129,2.90,7.0,9.0,Beginner,Diamond Peak,NV,1,green,1,1
1119,Beginner Terrain,8117.0,8100.0,17.0,120.00,140,0.40,7.0,9.0,Beginner,Beaver Creek,CO,1,green,1,1
323,Summer Road,9656.0,9640.0,16.0,294.00,23,0.20,5.0,8.0,Intermediate,Eldora,CO,1,blue,4,2
231,Slingshot,9843.0,9834.0,9.0,145.00,92,0.30,6.0,6.0,Beginner,Copper,CO,1,green,1,1


In [308]:
sub_df = df[df['resort'] == 'Winter Park']

In [233]:
list(sub_df['trail_name']);

In [296]:
cs = cosine_similarity(X[901].reshape(1,-1), X)[0]
css = enumerate(cs)
srtd = sorted(css, key=lambda x: x[1])[::-1]
srtd[-5:]

[(1007, 0.35797404937963168),
 (1039, 0.34264862662967777),
 (751, 0.22153939111139698),
 (1042, 0.21063989937351618),
 (750, 0.16996737448549881)]

In [297]:
df.iloc[[901,750,1042,751,1039,1007]]

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
901,Whistle Stop,10800.0,10640.0,160.0,1876.64,79,4.74,9.0,13.0,Novice,Winter Park,CO,1,green,2,1
750,Teaching Terrain,9407.0,9398.0,9.0,124.00,960,2.70,8.0,10.0,Beginner,Crested Butte,CO,1,green,1,1
1042,Gold Rush,9078.0,8898.0,180.0,428.00,2566,25.20,46.0,48.0,Advanced,Beaver Creek,CO,0,blue,5,2
751,Teaching Terrain,9360.0,9355.0,5.0,103.00,562,1.30,5.0,11.0,Beginner,Crested Butte,CO,1,green,1,1
1039,Dally Alley,8911.0,8766.0,144.0,607.00,1939,27.00,25.0,32.0,Low Intermediate,Beaver Creek,CO,0,blue,3,2
1007,Alphabet Chutes,11700.0,11350.0,350.0,694.98,2117.26,33.78,58.0,236.0,Expert,Winter Park,CO,0,bb,6,4


In [ ]:
index = df.index[(df['trail_name'] == trail_name) & (df['resort'] == resort_name)][0]